## New Dark Matter (DM) Simulation 
Using model from ... 

In [34]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from scipy.integrate import nquad, dblquad

np.set_printoptions(threshold=10)

In [53]:
class DMModel():
    def __init__(self, g = 6.67e-11, dmMass = 100, galaxyRadius = 5.2e20):
        self.g = g
        self.dmMass = dmMass
        self.galaxyRadius = galaxyRadius

    def DensityFunction(self, r, theta):
        # Function for radius, dependent on radius from galactic center and angle to the vertical axis.
        # Original function is in 2D
        meanDensity = (1/3) * 3e-28
        n = 1

        v0 = np.sqrt(4 * np.pi * self.g * meanDensity * self.galaxyRadius**2)
        pTheta = (1 + np.cos(theta))**(n + 1) + (1 - np.cos(theta))**(n + 1)
        sTheta = (4 * (n + 1) * np.sin(theta)**(2 * n))/(pTheta)**2
        rDensity = v0**2 / (4 * np.pi * self.g * r**2)
        density = rDensity * sTheta
        return density
    
    def SectionGalaxy(self, numRegions = 100):
        # Section galaxy into regions, numRegions^3 for each direction.
        # Store rMin, rMax, thetaMin, thetaMax, phiMin, phiMax for each region.
        rArray = np.linspace(0, self.galaxyRadius, numRegions)
        thetaArray = np.linspace(0, np.pi, numRegions)
        phiArray = np.linspace(0, 2*np.pi, numRegions)

        self.regions = []
        for i in range(numRegions - 1):
            for j in range(numRegions - 1):
                for k in range(numRegions - 1):
                    rMin, rMax = rArray[i], rArray[i + 1]
                    thetaMin, thetaMax = thetaArray[i], thetaArray[i + 1]
                    phiMin, phiMax = phiArray[i], phiArray[i + 1]

                    self.regions.append((rMin, rMax, thetaMin, thetaMax, phiMin, phiMax))

        # rMin = rArray[:-1]
        # rMax = rArray[1:]

        # thetaMin = thetaArray[:-1]
        # thetaMax = thetaArray[1:]

        # phiMin = phiArray[:-1]
        # phiMax = phiArray[1:]

        # Rmin, Tmin, Pmin = np.meshgrid(rMin, thetaMin, phiMin, indexing='ij')
        # Rmax, Tmax, Pmax = np.meshgrid(rMax, thetaMax, phiMax, indexing='ij')

        # self.regions = np.stack([Rmin, Rmax, Tmin, Tmax, Pmin, Pmax], axis=-1).reshape(-1, 6)
        # np array [rMin, rMax, thetaMin, thetaMax, phiMin, phiMax] per row

    def MonteCarloIntegral(self, N, region):
        # Random array of points within space for r, theta and phi.
        r = np.random.uniform(region[0], region[1], N)
        theta = np.random.uniform(region[2], region[3], N)
        # phi = np.random.uniform(region[4], region[5], N)

        regionVolume = (
            (region[1]**3 - region[0]**3)/3 *
            (np.cos(region[2]) - np.cos(region[3])) *
            (region[5] - region[4])
        )
        values = self.DensityFunction(r, theta)
        integrand = values * r**2 * np.sin(theta)
        
        return regionVolume * np.mean(integrand)


    def ConvertCoordinates(self):
        # Convert coordinates from cartesian to spherical for easier mass calculation.
        return

    def GetDMCount(self):
        # Integrate over each region of the galaxy to get total mass.
        #
        # Then divide total mass by DM Mass to get N for number of particles.
        self.regionMasses = np.array([
            self.MonteCarloIntegral(500, region) for region in self.regions
        ])

        self.numDM = self.regionMasses / self.dmMass
        print(self.numDM)
        return [self.regionMasses, self.numDM]
    
    def PopulateDMGalaxy(self):
        # With number of particles per region, now it's time to create a matrix with all DM particles.
        #
        # For decimals, summon as many as the last whole integer and subtract that from the total number of particles.
        # With the remaining decimal (<1), invert the number (1/decimal) and pick a random number between 1 and the nearest integer.
        # If the random number is 1 then an additional particle is summoned, otherwise it is not.
        return
    
    def GetRotationCurve(self, N = 10):
        # Summons N randomly placed test particles in the galaxy of DM.
        # Find the total mass of DM particles inside the radius of the test particles, 
        # then use that to find the orbital velocity required to have a stable circular orbit at that radius.
        return
    

In [54]:
model = DMModel()
model.SectionGalaxy(numRegions=100)

In [58]:
dmMasses = model.GetDMCount()

[1.61532730e+57 1.77149294e+57 1.75548182e+57 ... 4.65553402e+61
 4.63382317e+61 4.85236114e+61]


In [59]:
print("DM count: ", dmMasses[1][200000])
print("Region total mass: ", dmMasses[0][200000])

DM count:  3.3463561249002923e+66
Region total mass:  3.3463561249002923e+68
